In [1]:
assert "SENTINEL" not in locals(), "Kernel did not actually restart."
SENTINEL = "whatever"

import re
import sys
import torch
import transformers
import rich

sys.path.append("..")
import general_utils as utils # type: ignore

DEVICE = "cuda"

model_name = "distilgpt2"
model = transformers.AutoModelForCausalLM.from_pretrained(model_name).to(DEVICE)


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please use this form: https://docs.google.com/forms/d/e/1FAIpQLScPB8emS3Thkp66nvqwmjTEgxp8Y9ufuWTzFyr9kJ5AoI47dQ/viewform?usp=sf_link
CUDA SETUP: CUDA path found: /cvmfs/ai.mila.quebec/apps/x86_64/common/cuda/11.1/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.0
CUDA_SETUP: Detected CUDA version 111
CUDA_SETUP: Loading binary /home/mila/g/gagnonju/.main/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda111.so...


In [5]:
def setup_tokenizer(model_name, model):
    tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)
    utils.setattr_must_exist(tokenizer, "padding_side", "left")
    utils.setattr_must_exist(tokenizer, "pad_token", tokenizer.eos_token)
    utils.setattr_must_exist(model.config, "pad_token_id", model.config.eos_token_id)
    return tokenizer


tokenizer = setup_tokenizer(model_name, model)
kwargs = dict(
    constraints=None,
    # use_cache=True,
    repetition_penalty=None,
    do_sample=False,
    num_beams=1,
    max_new_tokens=100,
    min_length=0,
    jules_verbose=True,
)
encoded = tokenizer([
        "Gorbachev is", 
        "Grab popcorn, and let's go",
    ], 
    return_tensors="pt", 
    # add_special_tokens=False, 
    padding=True,
)

encoded_cuda = {k: v.to(DEVICE) for k, v in encoded.items()}



Using pad_token, but it is not set yet.


In [6]:
torch.all(encoded_cuda["input_ids"] < model.resize_token_embeddings().weight.shape[0])

for k, v in {k: getattr(tokenizer, k) for k in dir(tokenizer) if "token_id" in k and getattr(tokenizer, k) is not None}.items():
    print(k, v)

bos_token_id 50256
eos_token_id 50256
pad_token_id 50256
unk_token_id 50256


In [7]:

outputs = model.generate(
    inputs=encoded_cuda["input_ids"], 
    attention_mask=encoded_cuda["attention_mask"], 
    **kwargs
)

for output in outputs:
    decoded = tokenizer.decode(output).strip()
    cleaned = re.sub(r"\s+", " ", decoded)
    print(output.shape)
    rich.print(cleaned)

Generate logits_processor:
[]

Generate stopping critieria:
[<transformers.generation_stopping_criteria.MaxLengthCriteria object at 0x7efc38a6fcd0>]

 Doing greedy search

{
    'input_ids': tensor([[50256, 50256, 50256,    38, 27688, 42961,   318],
        [48400, 38914,    11,   290,  1309,   338,   467]], device='cuda:0'),
    'logits_processor': [],
    'stopping_criteria': [
        <transformers.generation_stopping_criteria.MaxLengthCriteria object at 
0x7efc38a6fcd0>
    ],
    'pad_token_id': 50256,
    'eos_token_id': 50256,
    'output_scores': False,
    'return_dict_in_generate': False,
    'synced_gpus': False,
    'attention_mask': tensor([[0, 0, 0, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1]], device='cuda:0'),
    'output_attentions': False,
    'output_hidden_states': False,
    'use_cache': None
}

torch.Size([107])


<|endoftext|><|endoftext|><|endoftext|>Gorbachev is a former Soviet Union member of the 
Soviet Union. He was a member of the Soviet Union's Soviet Union's Soviet Union's Soviet 
Union's Soviet Union's Soviet Union's Soviet Union's Soviet Union's Soviet Union's Soviet 
Union's Soviet Union's Soviet Union's Soviet Union's Soviet Union's Soviet Union's Soviet 
Union's Soviet Union's Soviet Union's Soviet Union's Soviet Union's Soviet Union's Soviet 
Union's Soviet Union's Soviet Union's Soviet Union's Soviet Union's Soviet Union's Soviet 
Union's

torch.Size([107])


Grab popcorn, and let's go!